In [1]:
import numpy as np

In [2]:
import pandas as pd

In [3]:
import redis

In [6]:
def listen_to_redis_port(redis_host = "localhost",
                         redis_port = 6379,
                         channel_name = ["1502_json",
                                                    "1501_json"]):
    
    redis_client = redis.Redis(host = redis_host,
                        port = redis_port)
    
    pubsub = redis_client.pubsub()
    pubsub.subscribe(channel_name[0])
    pubsub.subscribe(channel_name[1])

    for message in pubsub.listen():
        if message["type"] == "message":
            print(fr"recieved data : {message['data']}")




In [1]:
import requests
import zipfile
import io

date_str = '02092024'
url = f'https://www1.nseindia.com/content/historical/EQUITIES/2024/SEP/cm{date_str}bhav.csv.zip'

response = requests.get(url)

if response.status_code == 200:
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        z.extractall('bhavcopy')  # Extracts to a folder named 'bhavcopy'
        print("Bhavcopy downloaded and extracted successfully.")
else:
    print("Failed to download Bhavcopy.")


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [3]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.nseindia.com/products-services/equity-market-timings-holidays'
headers = {'User-Agent': 'Mozilla/5.0'}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

holidays = []

table = soup.find('table')
if table:
    rows = table.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        if cols:
            holiday_date = cols[0].text.strip()
            holiday_name = cols[1].text.strip()
            holidays.append((holiday_date, holiday_name))

print("Upcoming NSE Holidays:")
for holiday in holidays:
    print(f"{holiday[0]}: {holiday[1]}")


Upcoming NSE Holidays:


In [6]:
import pdfplumber
import pandas as pd

holidays = []

with pdfplumber.open('NseHolidays.pdf') as pdf:
    for page in pdf.pages:
        text = page.extract_text()

        lines = text.split('\n')
        for line in lines:
            if '2024' in line:  
                parts = line.split()
                date = " ".join(parts[:3])
                day = parts[3]
                description = " ".join(parts[4:])
                holidays.append({"Date": date, "Day": day, "Description": description})

df_holidays = pd.DataFrame(holidays)

print(df_holidays)

df_holidays.to_csv('nse_holidays_2024.csv', index=False)


                     Date       Day  \
0    Trading holidays for       the   
1     Market Segment, the  Exchange   
2           1 January 26,      2024   
3             2 March 08,      2024   
4             3 March 25,      2024   
5             4 March 29,      2024   
6             5 April 11,      2024   
7             6 April 17,      2024   
8               7 May 01,      2024   
9              8 June 17,      2024   
10             9 July 17,      2024   
11          10 August 15,      2024   
12         11 October 02,      2024   
13        12 November 01,      2024   
14        13 November 15,      2024   
15        14 December 25,      2024   
16            1 April 14,      2024   
17            2 April 21,      2024   
18        3 September 07,      2024   
19          4 October 12,      2024   
20         5 November 02,      2024   
21  *Muhurat Trading will        be   

                                          Description  
0                                  calendar ye

In [7]:
df_holidays

,Date,Day,Description
0,Trading holidays for,the,calendar year 2024
1,"Market Segment, the",Exchange,hereby notifies trading holidays for the calen...
2,"1 January 26,",2024,Friday Republic Day
3,"2 March 08,",2024,Friday Mahashivratri
4,"3 March 25,",2024,Monday Holi
5,"4 March 29,",2024,Friday Good Friday
6,"5 April 11,",2024,Thursday Id-Ul-Fitr (Ramadan Eid)
7,"6 April 17,",2024,Wednesday Shri Ram Navmi
8,"7 May 01,",2024,Wednesday Maharashtra Day
9,"8 June 17,",2024,Monday Bakri Id


In [ ]:
import pandas as pd


df['timestamp'] = pd.to_datetime(df['timestamp'])

df = df.sort_values('timestamp')

df['time_diff'] = df['timestamp'].diff().dt.total_seconds()

df['weighted_price'] = df['price'] * df['time_diff']

twap = df['weighted_price'].sum() / df['time_diff'].sum()